# Estadísticas Clasificación de Camiones

Importamos las librerias necesarias y cambiamos al directorio de trabajo

In [90]:
import os
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import ast

# Directorio de trabajo
SLIM_DIR='/root/src/models/research/slim'
os.chdir(SLIM_DIR)

Definimos funciones auxiliares

In [91]:
def classification_model_stats(TRAIN_DIR, DATASET_DIR):

    result = !python eval_image_classifier.py \
      --checkpoint_path={TRAIN_DIR}/all \
      --eval_dir={TRAIN_DIR}/all \
      --dataset_name=trucks \
      --dataset_split_name=validation \
      --dataset_dir={DATASET_DIR} \
      --model_name=inception_v3

    result2 = !python eval_image_classifier_cm.py  \
      --checkpoint_path={TRAIN_DIR}/all \
      --eval_dir={TRAIN_DIR}/all \
      --dataset_name=trucks \
      --dataset_split_name=validation \
      --dataset_dir={DATASET_DIR} \
      --model_name=inception_v3
    
    aux = [ s for s in result if 'Accuracy' in s][0]
    accuracy = float(aux[aux.find('Accuracy')+len('Accuracy')+1:-1])
    aux = [ s for s in result if 'FalsePositives' in s][0]
    fp = float(aux[aux.find('FalsePositives')+len('FalsePositives')+1:-1])
    aux = [ s for s in result if 'FalseNegatives' in s][0]
    fn = float(aux[aux.find('FalseNegatives')+len('FalseNegatives')+1:-1])
    aux = [ s for s in result if 'TruePositives' in s][0]
    tp = float(aux[aux.find('TruePositives')+len('TruePositives')+1:-1])
    aux = [ s for s in result if 'TrueNegatives' in s][0]
    tn = float(aux[aux.find('TrueNegatives')+len('TrueNegatives')+1:-1])

    labels = ! cat {DATASET_DIR}/tfrecords/labels.txt
    labels = [s[2:] for s in labels]

    cm = ','.join(result2[-1*len(labels):]).replace(', ',',').replace(' ',',').replace(',,',',').replace(',,',',').replace('[,','[')
    cm = ast.literal_eval(cm)

    df = pd.DataFrame({'Actual': labels})
    df = pd.concat([df, pd.DataFrame(cm, columns=labels)], axis=1)

    print("")
    print('Estadísticas Modelo___________________')
    print("Accuracy        : {0:.3f}".format(accuracy))
    print("True Positives  : {}".format(int(tp)))
    print("True Negatives  : {}".format(int(tn)))
    print("False Positives : {}".format(int(fp)))
    print("False Negatives : {}".format(int(fn)))
    print("")
    print("")

    print('Matriz de confusion___________________')
    display(df.set_index("Actual"))


## Trucks

Base de datos con 5 clases de camiones: concrete-mixer, crane-truck, dump-truck, other-truck, water-tank-truck

In [93]:
# Donde esta el modelo entrenado (checkpoint)
TRAIN_DIR='/root/models/tf-slim/train/Trucks/inception_v3'
# Donde esta la base de datos
DATASET_DIR='/root/databases/Trucks'

classification_model_stats(TRAIN_DIR, DATASET_DIR)


Estadísticas Modelo___________________
Accuracy        : 0.928
True Positives  : 465
True Negatives  : 125
False Positives : 5
False Negatives : 5


Matriz de confusion___________________


,concrete-mixer,crane-truck,dump-truck,other-truck,water-tank-truck
Actual,,,,,
concrete-mixer,125,0,1,0,4
crane-truck,0,165,12,0,1
dump-truck,1,3,184,0,5
other-truck,2,0,5,2,1
water-tank-truck,2,2,4,0,81


## Trucks-concrete

Base de datos con 2 clases de camiones: concrete-mixer, no-concrete-mixer

In [94]:
TRAIN_DIR='/root/models/tf-slim/train/Trucks-concrete/inception_v3'
DATASET_DIR='/root/databases/Trucks-concrete'

classification_model_stats(TRAIN_DIR, DATASET_DIR)


Estadísticas Modelo___________________
Accuracy        : 0.999
True Positives  : 1438
True Negatives  : 361
False Positives : 1
False Negatives : 0


Matriz de confusion___________________


,concrete-truck,no-concrete-truck
Actual,,
concrete-truck,361,1
no-concrete-truck,0,1438


## Trucks4

Base de datos con 4 clases de camiones: concrete-mixer, crane-truck, dump-truck, water-tank-truck

In [95]:
# Donde esta el modelo entrenado (checkpoint)
TRAIN_DIR='/root/models/tf-slim/train/Trucks4/inception_v3'
# Donde esta la base de datos
DATASET_DIR='/root/databases/Trucks4'

classification_model_stats(TRAIN_DIR, DATASET_DIR)


Estadísticas Modelo___________________
Accuracy        : 0.973
True Positives  : 445
True Negatives  : 147
False Positives : 8
False Negatives : 0


Matriz de confusion___________________


,concrete-mixer,crane-truck,dump-truck,water-tank-truck
Actual,,,,
concrete-mixer,146,0,1,7
crane-truck,0,147,4,0
dump-truck,0,2,206,2
water-tank-truck,0,0,0,85


In [73]:
from pandas_ml import ConfusionMatrix

y_actu = [2, 0, 2, 2, 0, 1, 1, 2, 2, 0, 1, 2]
y_pred = [0, 0, 2, 1, 0, 2, 1, 0, 2, 0, 2, 2]
cm = ConfusionMatrix(y_actu, y_pred)
cm

Predicted  0  1  2  __all__
Actual                     
0          3  0  0        3
1          0  1  2        3
2          2  1  3        6
__all__    5  2  5       12